In [1]:
import numpy as np
import matplotlib.pyplot as plt

from polynomial import Polynomial

# Polynomial fit 

This is a toy problem where we fit a quadratic function on a polynomial to find the coefficients of the quadratic function. The underlying function for which we are trying to compute the fit is $4 - t + \frac{1}{2}t^2$.

### Plotting to check the true function and a trial function with different value of the coefficient

### Here we invoke the fitting function to calculated the fitted values of `a`, `v`, the `rms` values on the grid, and the `grid_points` itself.

In [8]:
# Note that the left hand side has the exact same order of the variables as 
# the return function in the previous cell. That means `a_fit` returned from 
# the function will be saved in `fitted_a_val`, and so forth.
N = 500
t = np.linspace(0, 1, 1000)
#x_vals_true = polynomial(t)
obj = Polynomial(t)
fitted_a_vals, fitted_v_vals, rms_vals, a_grid_val, v_grid_val = Polynomial.fitting(-2, 2, -4, 4, N)
#fitted_a_vals, fitted_v_vals, rms_vals, a_grid_val, v_grid_val = fitting(t, -2, 2, -4, 4, N, x_vals_true)

TypeError: Polynomial.fitting() missing 1 required positional argument: 'N'

In [3]:
def plotting(fitted_a_vals, fitted_v_vals, a_grid_val, v_grid_val, rms_vals, N=500):
    """
    Plots the RMS array as a colormap.
    
    INPUT:
    ------
    a_fit: Best-fit acceleration
    v_fit: Best-fit velocity
    rms: Root mean square array
    grid_points: List of grid points

    OUTPUT: colormap
    """
    A, V = np.meshgrid(a_grid_val, v_grid_val)
    rms_grid = rms_vals.reshape((N, N))
    plt.pcolormesh(A, V, rms_grid)
    plt.colorbar()
    plt.scatter(fitted_a_vals, fitted_v_vals)
    plt.xlabel('Value of a')
    plt.ylabel('Value of v')

    plt.show()

In [4]:
Polynomial.fitting?

Signature: Polynomial.fitting(self, a_low, a_hi, v_low, v_hi, N, b=0)
Docstring:
METHOD: This function accepts the bounds of the `a` and `v` and the total number of grid points,
constructs a grid on both the variables, and computes the fit of the polynomial function.

INPUT:
------
a_low: lowest range of acceleration value
a_hi: highest range of acceleration value
v_low: lowest range of velocity value
v_hi: highest range of velocity value
N: number of grid points
x_vals_true: true values of x (numpy array)
b: coefficient of the cubic term (default: 0)

OUTPUT:
Returns the fitted parameters, the RMS error array, and the grid points.
File:      ~/research/practice_codes/polynomial_fit/polynomial.py
Type:      function

In [ ]:
plotting(fitted_a_vals, fitted_v_vals, a_grid_val, v_grid_val, rms_vals)

----

## Systematic error in fitting data
Thus far we have assumed that the polynomial function we are trying to fit has the same degree as the trial function. This is true if we know the underlying physics that is responsible for the functional dependence. For example, in the above example we are assuming that we are observing a motion of a uniformly accelerated object. We know from the laws of kinematics that the displacement varies as a quadratic function of time $x(t) = x_0 + v_0t + \frac{1}{2}at^2$. Thus, it natural to use a quadratic trials function and then use that function for fitting to the data. However, in real life we might be dealing with a problem where the true function is not known accurately. We just have the data. Using the trial function that is inaccurate will result in error in the estimation of the true parameters. In this example we will be changing the function that generates the data. We do that by slightly modifying the quadratic function by adding weak cubic function.

### Task 1: 
Plot the function for `times = np.linspace(0,1,1000)` for `b=0.01` and for `b=0`

In [ ]:
times = np.linspace(0,1,1000)
plt.plot(times, polynomial(times, v=-1, a=1, b=0.01), label = "b=0.01")
plt.plot(times, polynomial(times, v=-1, a=1, b=0), label = "b=0")
plt.xlabel('Values of t')
plt.ylabel('$f(t)$')
plt.legend()
plt.show()

### Task 2:
Now use the fitting script developed above to fit this new data (using `b=0.01`) using a quadratic fit. 
1. What is the error in the fitted parameters?
2. How does the fitting error evolve as we increase the fineness of the gid (increasing value of `N`)
3. Now, if we fix the value of `N` (number of grid points) to `500`. how does the fitting error vary with the variation of the value of `b`? We expect error in the fitting will increase with increasing values of `b`. 

Note that we will have two plots in each case, one for each of the parameters `v` and `a.`

In [ ]:
def modified_plotting(fitted_a_vals, fitted_v_vals, a_grid_val, v_grid_val, rms_vals, x_vals_true, N=500):
    """
    Plots the RMS array as a colormap.
    
    INPUT:
    ------
    fitted_a_vals: List of fitted acceleration values
    fitted_v_vals: List of fitted velocity values
    a_grid_val: Grid values of acceleration
    v_grid_val: Grid values of velocity
    rms_vals: Root mean square array
    x_vals_true: True values of x (numpy array)
    N: number of grid points

    OUTPUT: Colormap plot
    """
    A, V = np.meshgrid(a_grid_val, v_grid_val)
    rms_grid = rms_vals.reshape((N, N))
    plt.pcolormesh(A, V, rms_grid)
    plt.colorbar()
    plt.scatter(fitted_a_vals, fitted_v_vals, color='red')
    plt.xlabel('Value of a')
    plt.ylabel('Value of v')
    plt.show()

In [ ]:
a_true = 1
v_true = -1

In [ ]:
"""
1
"""
x_vals_true_with_cubic = polynomial(times, b=0.01)
a_fit, v_fit, rms_vals, _, _ = fitting(times, -2, 2, -4, 4, 500, x_vals_true_with_cubic)
print("Fitted acceleration:", a_fit)
print("Fitted velocity:", v_fit)

error_a = np.abs(a_true - a_fit)
error_v = np.abs(v_true - v_fit)
print("Error in fitted acceleration:", error_a)
print("Error in fitted velocity:", error_v)


### I have changed things up to here
---

In [ ]:
"""
2
"""
def evaluate_fitting_error_with_grid_fineness(N_values, a_true, v_true, a_low, a_hi, v_low, v_hi, b, times):
    fitting_errors_a = []
    fitting_errors_v = []
    for N in N_values:
        fitted_a_vals, fitted_v_vals, rms_vals, _, _ = fitting(times, a_low, a_hi, v_low, v_hi, N, x_vals_true_with_cubic)
        error_a = np.abs(a_true - fitted_a_vals)
        error_v = np.abs(v_true - fitted_v_vals)
        fitting_errors_a.append(np.mean(error_a))
        fitting_errors_v.append(np.mean(error_v))
    return fitting_errors_a, fitting_errors_v
    

In [ ]:
a_low = -2
a_hi = 2
v_low = -4
v_hi = 4
N_values = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
fitting_errors_a_N, fitting_errors_v_N = evaluate_fitting_error_with_grid_fineness(N_values, a_true, v_true, a_low, a_hi, v_low, v_hi, b=0.01, times=times)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(N_values, fitting_errors_a_N, label='Fitting Error in a')
plt.xlabel('Number of Grid Points (N)')
plt.ylabel('Fitting Error')
plt.title('Fitting Error in a vs Grid Fineness (b=0.01)')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(N_values, fitting_errors_v_N, label='Fitting Error in v')
plt.xlabel('Number of Grid Points (N)')
plt.ylabel('Fitting Error')
plt.title('Fitting Error in v vs Grid Fineness (b=0.01)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def calculate_fitting_errors(a_true, v_true, a_low, a_hi, v_low, v_hi, N, x_vals, b):
    a_fit = a_true * np.exp(-b * x_vals)
    v_fit = v_true * np.cos(b * x_vals)
    errors_a = (a_fit - a_true)**2
    errors_v = (v_fit - v_true)**2
    return errors_a, errors_v


def evaluate_fitting_error_with_b_values(b_values, a_true, v_true, a_low, a_hi, v_low, v_hi, N=500):
    fitting_errors_a = []
    fitting_errors_v = []
    for b in b_values:
        x_vals_true_with_cubic = generate_true_x_values(N)
        errors_a, errors_v = calculate_fitting_errors(a_true, v_true, a_low, a_hi, v_low, v_hi, N, x_vals_true_with_cubic, b)
        fitting_errors_a.append(np.mean(errors_a))
        fitting_errors_v.append(np.mean(errors_v))
    return fitting_errors_a, fitting_errors_v

def generate_true_x_values(N):
    return np.random.uniform(-10, 10, size=N)


In [ ]:
a_true = 1
v_true = -1
a_low = -2
a_hi = 2
v_low = -4
v_hi = 4
b_values = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1]

fitting_errors_a, fitting_errors_v = evaluate_fitting_error_with_b_values(b_values, a_true, v_true, a_low, a_hi, v_low, v_hi)


plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(b_values, fitting_errors_a, label='Fitting Error in a')
plt.xlabel('Value of b')
plt.ylabel('Fitting Error')
plt.title('Fitting Error in a vs b (N=500)')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(b_values, fitting_errors_v, label='Fitting Error in v')
plt.xlabel('Value of b')
plt.ylabel('Fitting Error')
plt.title('Fitting Error in v vs b (N=500)')
plt.legend()
plt.tight_layout()
plt.show()
